In [6]:
## Installation of required libraries
!apt-get install poppler-utils
!pip install tesseract libmagic pandoc
!sudo apt-get install libleptonica-dev tesseract-ocr tesseract-ocr-dev libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn
!sudo apt-get install tesseract-ocr


'apt-get' is not recognized as an internal or external command,
operable program or batch file.


'sudo' is not recognized as an internal or external command,
operable program or batch file.
'sudo' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
%pip install "unstructured[all-docs]" unstructured-client watermark

  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached zipp-3.21.0-py3-none-any.whl.metadata (3.7 kB)
Using cached zipp-3.21.0-py3-none-any.whl (9.6 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.pptx import partition_pptx

| **Parameter**                  | **Type**              | **Description**                                                                                  |
|--------------------------------|-----------------------|--------------------------------------------------------------------------------------------------|
| `filename`                     | `str | None`          | Path to the PDF file. Specify if the PDF is stored locally.                                      |
| `file`                         | `IO[bytes] | None`    | A binary file object if the PDF is already loaded in memory.                                    |
| `include_page_breaks`          | `bool`               | Whether to include page breaks as elements in the output. Default is `False`.                   |
| `strategy`                     | `str`                | Extraction strategy: `"hi_res"`, `"ocr_only"`, `"fast"`, or `"auto"`. Default is `"auto"`.      |
| `infer_table_structure`        | `bool`               | If `True`, preserves table structure as HTML. Only applies to `strategy="hi_res"`.              |
| `ocr_languages`                | `str | None`          | Specifies OCR languages if needed.                                                             |
| `languages`                    | `list[str] | None`    | List of languages present in the document.                                                     |
| `metadata_filename`            | `str | None`          | Metadata field for the source filename.                                                        |
| `metadata_last_modified`       | `str | None`          | Metadata field for the document’s last modified date.                                           |
| `chunking_strategy`            | `str | None`          | Determines how text is divided into smaller or larger elements.                                 |
| `hi_res_model_name`            | `str | None`          | Specifies the layout detection model for `strategy="hi_res"`.                                   |
| `extract_images_in_pdf`        | `bool`               | Whether to extract images. Only applies to `strategy="hi_res"`.                                 |
| `extract_image_block_types`    | `list[str] | None`    | Specifies the types of image blocks to extract (e.g., `["Image", "Table"]`).                    |
| `extract_image_block_output_dir`| `str | None`         | Directory to save extracted images. Only applies to `strategy="hi_res"`.                        |
| `extract_image_block_to_payload`| `bool`              | If `True`, encodes images as base64 data in metadata fields.                                    |
| `starting_page_number`         | `int`                | Page number to start extraction. Default is `1`.                                               |
| `extract_forms`                | `bool`               | If `True`, extracts form elements (e.g., key-value pairs).                                      |
| `form_extraction_skip_tables`  | `bool`               | If `True`, skips regions designated as tables during form extraction. Default is `True`.        |


In [5]:
## define the function for extraction of table image and texts from a pdf

def extract_and_categorize(pdf_path):
    """
    Extracts and categorizes elements from a PDF.

    Args:
        pdf_path (str): Path to the PDF file.

    Returns:
        dict: Dictionary categorizing elements by their types.
    """
    # Extract elements using partition_pdf
    elements = partition_pdf(
        filename=pdf_path,
        include_page_breaks=True,
        strategy="auto",  # Automatically chooses between "fast" and "hi_res"
        infer_table_structure=True,
        extract_images_in_pdf=True,
        extract_image_block_types=["Image", "Table"],
        starting_page_number=1,
        extract_forms=False,
        form_extraction_skip_tables=False
    )

    return elements

# Path to your PDF
pdf_path = "example.pdf"

# Extract and categorize PDF content
categorized_elements = extract_and_categorize(pdf_path)


PDFPageCountError: Unable to get page count.
I/O Error: Couldn't open file 'example.pdf': No error.


In [45]:
## convert the elements into dict and categorize them
content = [el.to_dict() for el in categorized_elements]

In [46]:
cat_con = {}
for one in content:
  try:
    cat_con[one['type']].append(one)
  except:
    cat_con[one['type']] = [one]

In [47]:
cat_con

{'Image': [{'type': 'Image',
   'element_id': '32963ad8ebbe17ed1adf8e7069a32eb6',
   'text': 'wy) ACCELIO..',
   'metadata': {'coordinates': {'points': ((160.09972222222223,
       99.99138888888884),
      (160.09972222222223, 233.9802777777777),
      (357.08333333333337, 233.9802777777777),
      (357.08333333333337, 99.99138888888884)),
     'system': 'PixelSpace',
     'layout_width': 1700,
     'layout_height': 2200},
    'last_modified': '2024-12-25T15:47:17',
    'filetype': 'application/pdf',
    'languages': ['eng'],
    'page_number': 1,
    'image_path': '/content/figures/figure-1-1.jpg',
    'filename': 'example.pdf'}},
  {'type': 'Image',
   'element_id': '2d1ba962349d0889d32fe25d077f6019',
   'text': 'Acrobat Reader - [ap_bookmark. pdf] E] File Edit Document View Window Help NOeSS ES hi «> «> DOR 4S a [[ Bookmarks “thumbnais [P| Date 2000-01-1 Description Description for item # 1 Type TYPE1 Amount 11.00',
   'metadata': {'coordinates': {'points': ((160.09972222222223,
  

In [48]:
## types of elements present
cat_con.keys()

dict_keys(['Image', 'Title', 'Table', 'NarrativeText', 'PageBreak', 'ListItem'])

In [49]:
partition_pptx?

### Parameters for `partition_pptx`

| Parameter              | Type             | Description                                                                                                                                                                      |
|------------------------|------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `filename`             | `str \| None`   | Path to the PowerPoint file.                                                                                                                                                    |
| `file`                 | `IO[bytes] \| None` | File object opened in "rb" mode.                                                                                                                                                 |
| `include_page_breaks`  | `bool`          | If `True`, includes a `PageBreak` element between slides. Default is `True`.                                                                                                   |
| `include_slide_notes`  | `bool \| None`  | If `True`, extracts slide notes as elements.                                                                                                                                    |
| `infer_table_structure`| `bool`          | If `True`, extracted table elements include a `text_as_html` metadata field for rendering rows and cells as HTML. Regardless, the `"text"` field always contains unstructured table text. |
| `starting_page_number` | `int`           | Specifies the page number for the first slide. This metadata can be useful when the presentation is part of a larger document.                                                  |
| `strategy`             | `str`           | Extraction strategy. Options include `"fast"`. Default is `"fast"`.                                                                                                            |
| `**kwargs`             | `Any`           | Additional parameters for advanced configurations.                                                                                                                              |


In [50]:
from unstructured.partition.pptx import partition_pptx

# Extract from a file path
elements = partition_pptx(filename="example.pptx", include_slide_notes=True)

# Access elements
for element in elements:
    print(element.text)


Presentation Title
Author
Department
Date
Location

Cover slide option #1
Presentation Title
Author
Department
Date
Location

Cover slide option #3
Presentation Title
Author
Department
Date
Location

GraphTitle
Additional Notes - Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem

Chart Title
Column A B C D XXXXXXXX XX XX XX XXXXXXXX XX XX XX XXXXXXXX XX XX XX XXXXXXXX XX XX XX XXXXXXXX XX XX XX XXXXXXXX XX XX XX

List Title
Lorem ipsum dolor sit amet 
Aenean commodo ligula eget dolor 
Cum sociis natoque penatibus et magnis dis parturient montes 
Donec quam felis, ultricies nec, pellentesque eu 
Lorem ipsum dolor sit amet, consectetuer adipiscing elit 
Aenean massa 
Aenean commodo ligula eget dolor

Slide Title
Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commo

In [51]:
content = [el.to_dict() for el in elements]

content

[{'type': 'Title',
  'element_id': 'd9fb501c7d9a09d7b0cfd3028c1b67ad',
  'text': 'Presentation Title',
  'metadata': {'category_depth': 0,
   'filename': 'example.pptx',
   'last_modified': '2024-12-25T17:04:08',
   'page_number': 1,
   'languages': ['fra', 'cat'],
   'filetype': 'application/vnd.openxmlformats-officedocument.presentationml.presentation'}},
 {'type': 'Title',
  'element_id': 'b1f9dd782c2134c5741ebe629c1e5636',
  'text': 'Author',
  'metadata': {'category_depth': 1,
   'filename': 'example.pptx',
   'last_modified': '2024-12-25T17:04:08',
   'page_number': 1,
   'languages': ['fra', 'cat'],
   'filetype': 'application/vnd.openxmlformats-officedocument.presentationml.presentation',
   'parent_id': 'd9fb501c7d9a09d7b0cfd3028c1b67ad'}},
 {'type': 'Title',
  'element_id': 'b8856b2aedb64f1816a63ecaebf5e54f',
  'text': 'Department',
  'metadata': {'category_depth': 1,
   'filename': 'example.pptx',
   'last_modified': '2024-12-25T17:04:08',
   'page_number': 1,
   'languages'

In [52]:
cat_con = {}
for one in content:
  try:
    cat_con[one['type']].append(one)
  except:
    cat_con[one['type']] = [one]

In [53]:
## types of elements present
cat_con.keys()

dict_keys(['Title', 'PageBreak', 'NarrativeText', 'Table', 'ListItem'])

In [54]:
from unstructured.partition.xlsx import partition_xlsx

def extract_excel_content(file_path=None, file_obj=None):
    """
    Extracts text, tables, and images from an Excel (.xlsx) file.

    Parameters:
        file_path (str): Path to the Excel file.
        file_obj (IO[bytes]): File object of the Excel file.

    Returns:
        list: List of extracted document elements.
    """
    # Extract content from the Excel file
    elements = partition_xlsx(
        filename=file_path,
        file=file_obj,
        find_subtable=True,                # Partition subtables as separate elements
        include_header=True,               # Include headers in table metadata
        infer_table_structure=True,        # Preserve table structure as HTML
        starting_page_number=1             # Page number starts from 1
    )

    # Convert elements to a dictionary for easy access
    content = [el.to_dict() for el in elements]

    return content


In [55]:
content = extract_excel_content('/content/Sample.xlsx')

In [56]:
## categorizing the element into their respective categories
cat_con = {}
for one in content:
  try:
    cat_con[one['type']].append(one)
  except:
    cat_con[one['type']] = [one]

In [57]:
cat_con

{'NarrativeText': [{'type': 'NarrativeText',
   'element_id': '21f706432ed216dddea2d3bd734f86c3',
   'text': 'Link to worldometer',
   'metadata': {'file_directory': '/content',
    'filename': 'Sample.xlsx',
    'last_modified': '2024-12-25T17:26:35',
    'page_name': 'List1',
    'page_number': 1,
    'languages': ['eng'],
    'filetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'}}],
 'Table': [{'type': 'Table',
   'element_id': '1ab67dddef666d1523369baafffb1af3',
   'text': "Unnamed: 0 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9 Unnamed: 10 Unnamed: 11 Unnamed: 12 Unnamed: 13 Unnamed: 14 Unnamed: 15 Country Year Rank Total S1: Demographic Pressures S2: Refugees and IDPs C3: Group Grievance E3: Human Flight and Brain Drain E2: Economic Inequality E1: Economy P1: State Legitimacy P2: Public Services P3: Human Rights C1: Security Apparatus C2: Factionalized Elites X1: External Intervention Somalia 2023 1st